In [1]:
import lightgbm as lgb
__file__=''
import os,sys
import pandas as pd
sys.path.append(os.path.join(os.path.dirname(__file__),'../LIB/'))
sys.path.append(os.path.join(os.path.dirname(__file__),'../../../../automl/automl_libs/'))
from env import FILE
from sklearn.feature_extraction import FeatureHasher
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from tqdm import tqdm
from scipy import sparse 
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import chi2, SelectPercentile,SelectKBest
import gc
import time
from sklearn.feature_selection import chi2, SelectPercentile

In [2]:
from scipy.sparse import csr_matrix
def save_csr(matrix,file_prefix='csr'):
    data_file = file_prefix + '_data.npy'
    indices_file = file_prefix + '_indices.npy'
    indptr_file = file_prefix + '_indptr.npy'
    np.save(data_file,matrix.data)
    print('data save')
    np.save(indices_file,matrix.indices)
    print('indices save')
    np.save(indptr_file,matrix.indptr)
    print('indptr save')
    print('save done!')
    
def load_csr(file_prefix='csr'):
    data_file = file_prefix + '_data.npy'
    indices_file = file_prefix + '_indices.npy'
    indptr_file = file_prefix + '_indptr.npy'
    data = np.load(data_file)
    print('data load')
    indices = np.load(indices_file)
    print('indices load')
    indptr = np.load(indptr_file)
    print('indptr load')
    csr = csr_matrix((data, indices, indptr))
    return csr

In [3]:
train_csr = load_csr('../../data/features/xgb/train_csr')
predict_csr = load_csr('../../data/features/xgb/predict_csr')
train_y = np.load('../../data/features/xgb/train_y_f620_b70_col17k.npy')
predicted_file = pd.read_csv('../../data/features/xgb/predict_f620_b70_col17k.csv')

data load
indices load
indptr load
data load
indices load
indptr load


# Load More features

In [4]:
X_fe = pd.read_pickle('../../data/features/xgb/new_feats_838.pkl')
print(X_fe.shape)
X_ = X_fe.copy()

(3072915, 838)


In [5]:
cut_bin = 30
for col in tqdm(X_fe.columns):
        X_[col] = pd.cut(X_fe[col], bins=cut_bin, labels=False, retbins=True, right=False)[0]  

100%|██████████| 838/838 [15:20<00:00,  1.10s/it]


In [6]:
train_csr_ff = sparse.hstack(
    (sparse.csr_matrix(X_.iloc[:train_csr.shape[0]]), train_csr), 'csr').astype(
    'float32')
predict_csr_ff = sparse.hstack(
    (sparse.csr_matrix(X_.iloc[train_csr.shape[0]:]), predict_csr), 'csr').astype('float32')
print(train_csr_ff.shape)

NameError: name 'train_csr_f' is not defined

In [10]:
index_y1 = np.array(range(len(train_y)))[train_y==1]
df_posi = pd.DataFrame({'index_label':index_y1})
index_y0 = np.array(range(len(train_y)))[train_y==0]
df_nega = pd.DataFrame({'index_label':index_y0})
df_nega = df_nega.sample(len(df_posi))
df_label = pd.concat([df_posi,df_nega],sort=False)
df_label = df_label.sample(frac=1)
labels_index = df_label.index_label.values
print(len(labels_index))


1190804


In [16]:
train_csr_f = train_csr_ff[labels_index]
train_yy = train_y[labels_index]
predict_csr_f = predict_csr_ff

KeyboardInterrupt: 

In [17]:
import gc
gc.collect()

991

In [ ]:
# feature_select = SelectKBest(chi2, percentile=10000)
# feature_select.fit(train_csr, train_y)
# train_csr = feature_select.transform(train_csr)
# predict_csr = feature_select.transform(predict_csr)
# print('feature select')
# print(train_csr.shape)

# Generate sample

In [4]:
# sample_size = 400000
# sample_times = 2
# sample_all = pd.DataFrame({'data':range(train_csr.shape[0])})
# sample_index_list = []
# samples = []
# samples_y = []
# for i in tqdm(range(sample_times)):
#     sample_index_list.append(list(sample_all.sample(sample_size).index))
#     samples.append(train_csr[sample_index_list[i]])
#     samples_y.append(train_y[sample_index_list[i]])


100%|██████████| 2/2 [00:11<00:00,  5.58s/it]


# Define lgb model

In [ ]:
lgb_model = lgb.LGBMClassifier(
    boosting_type='gbdt', num_leaves=32, reg_alpha=0, reg_lambda=0.1,
    max_depth=-1, n_estimators=5000, objective='binary',
    subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
    learning_rate=0.05, random_state=1001, n_jobs=16
)


In [19]:
def test_on_sample(train_csr_f,train_y_f,lgb_model,n_fold = 5,seed=None,first=2500):
    if seed is None:
        seed = np.random.randint(1000)
    cv_list = []
    skf = StratifiedKFold(n_splits=n_fold, random_state=seed, shuffle=True)
    best_score = []


    feature_importance_tmp_split = []
    feature_importance_tmp_gain = []
    df_fea = pd.DataFrame()

    t1 = time.time()
    for index, (train_index, test_index) in enumerate(skf.split(train_csr_f, train_y_f)):
        lgb_model.fit(train_csr_f[train_index], train_y_f[train_index],
                      eval_set=[(train_csr_f[train_index], train_y_f[train_index]),
                                (train_csr_f[test_index], train_y_f[test_index])], early_stopping_rounds=200,verbose=200)
        best_score.append(lgb_model.best_score_['valid_1']['binary_logloss'])
        print(best_score)
        print('cv:',np.mean(best_score))
        feature_importance_tmp_split.append(lgb_model.booster_.feature_importance(importance_type='split'))
        feature_importance_tmp_gain.append(lgb_model.booster_.feature_importance(importance_type='gain'))

    df_fea['col_name'] = lgb_model.booster_.feature_name()
    df_fea['split'] = np.mean(feature_importance_tmp_split,axis=0)
    df_fea['gain'] = np.mean(feature_importance_tmp_gain,axis=0)

    cv_list.append(np.mean(best_score))
    t2 = time.time()
    print('time spend: {} s'.format(t2-t1))
    print('after dropping...')


    split_index = list(df_fea[df_fea.split!=0].index)
    gain_index = list(df_fea[df_fea.gain!=0].index)
    #union
#     retain_index = list(set(split_index+gain_index))
    #intersection
#     retain_index = list(set(split_index).intersection(set(gain_index)))
    if first is None:
        first = 500
    split_index = df_fea.sort_values('split',ascending=False)['split'].index.values[:first]
    gain_index = df_fea.sort_values('gain',ascending=False)['gain'].index.values[:first]
    retain_index = list(set(split_index).intersection(set(gain_index)))
    print(len(retain_index))

    best_scor2 = []
    t1 = time.time()
    for index, (train_index, test_index) in enumerate(skf.split(train_csr_f, train_y_f)):
        lgb_model.fit(train_csr_f[train_index][:,retain_index], train_y_f[train_index],
                      eval_set=[(train_csr_f[train_index][:,retain_index], train_y_f[train_index]),
                                (train_csr_f[test_index][:,retain_index], train_y_f[test_index])], early_stopping_rounds=200,verbose=200)
        best_scor2.append(lgb_model.best_score_['valid_1']['binary_logloss'])
        print(best_scor2)
        print('cv:',np.mean(best_scor2))
    cv_list.append(np.mean(best_scor2))
    drop = False
    if cv_list[-1] < cv_list[-2]:
        print('dropping successfully!')
        print('cv drop:{}'.format(cv_list[-2]-cv_list[-1]))
        drop = True
    else:
        print('dropping failed!!!!!!')
        print('cv increase:{}'.format(cv_list[-1]-cv_list[-2]))
    t2 = time.time()
    print('time spend: {} s'.format(t2-t1))
    return df_fea, retain_index, drop

In [20]:
def verify_on_sample(train_csr_f,train_y_f,lgb_model,retain_index,n_fold = 3,seed=None,cv_list=[]):
    if seed is None:
        seed = np.random.randint(1000)

    skf = StratifiedKFold(n_splits=n_fold, random_state=seed, shuffle=True)
    best_score = []
    
    if len(cv_list) == 0:

        t1 = time.time()
        for index, (train_index, test_index) in enumerate(skf.split(train_csr_f, train_y_f)):
            lgb_model.fit(train_csr_f[train_index], train_y_f[train_index],
                          eval_set=[(train_csr_f[train_index], train_y_f[train_index]),
                                    (train_csr_f[test_index], train_y_f[test_index])], early_stopping_rounds=100,verbose=200)
            best_score.append(lgb_model.best_score_['valid_1']['binary_logloss'])
            print(best_score)
            print('cv:',np.mean(best_score))




        cv_list.append(np.mean(best_score))
        t2 = time.time()
        print('time spend: {} s'.format(t2-t1))
        print('after dropping...')

        print(len(retain_index))

    best_scor2 = []
    t1 = time.time()
    for index, (train_index, test_index) in enumerate(skf.split(train_csr_f, train_y_f)):
        lgb_model.fit(train_csr_f[train_index][:,retain_index], train_y_f[train_index],
                      eval_set=[(train_csr_f[train_index][:,retain_index], train_y_f[train_index]),
                                (train_csr_f[test_index][:,retain_index], train_y_f[test_index])], early_stopping_rounds=100,verbose=200)
        best_scor2.append(lgb_model.best_score_['valid_1']['binary_logloss'])
        print(best_scor2)
        print('cv:',np.mean(best_scor2))
    cv_list.append(np.mean(best_scor2))
    t2 = time.time()
    print('time spend: {} s'.format(t2-t1))

    if cv_list[-1] < cv_list[-2]:
        print('dropping successfully!')
        print('cv drop:{}'.format(cv_list[-2]-cv_list[-1]))
        return True
    else:
        print('dropping failed!!!!!!')
        print('cv increase:{}'.format(cv_list[-1]-cv_list[-2]))
        return False
    
    

In [ ]:
df_fea, retain_index, ifdrop = test_on_sample(train_csr_f,train_yy,lgb_model,n_fold = 5,seed=19,first=2500)

Training until validation scores don't improve for 200 rounds.
[200]	valid_0's binary_logloss: 0.553569	valid_1's binary_logloss: 0.555471
[400]	valid_0's binary_logloss: 0.54943	valid_1's binary_logloss: 0.553944


In [25]:
if ifdrop:
    verify_drop = verify_on_sample(samples[1],samples_y[1],lgb_model,retain_index,n_fold = 5,seed=21)

Training until validation scores don't improve for 100 rounds.
[200]	valid_0's binary_logloss: 0.41176	valid_1's binary_logloss: 0.416903
[400]	valid_0's binary_logloss: 0.402715	valid_1's binary_logloss: 0.416445
[600]	valid_0's binary_logloss: 0.394742	valid_1's binary_logloss: 0.416382
Early stopping, best iteration is:
[507]	valid_0's binary_logloss: 0.398304	valid_1's binary_logloss: 0.416348
[0.41634779832398616]
cv: 0.41634779832398616
Training until validation scores don't improve for 100 rounds.
[200]	valid_0's binary_logloss: 0.410971	valid_1's binary_logloss: 0.419463
Early stopping, best iteration is:
[244]	valid_0's binary_logloss: 0.40881	valid_1's binary_logloss: 0.419309
[0.41634779832398616, 0.4193088064896909]
cv: 0.41782830240683855
Training until validation scores don't improve for 100 rounds.
[200]	valid_0's binary_logloss: 0.411046	valid_1's binary_logloss: 0.419822
Early stopping, best iteration is:
[284]	valid_0's binary_logloss: 0.406948	valid_1's binary_loglos

In [29]:
df_fea, retain_index, ifdrop = test_on_sample(train_csr,train_y,lgb_model,n_fold = 5,seed=19,first=1500)

Training until validation scores don't improve for 100 rounds.
[200]	valid_0's binary_logloss: 0.416214	valid_1's binary_logloss: 0.416022
[400]	valid_0's binary_logloss: 0.413937	valid_1's binary_logloss: 0.414851
[600]	valid_0's binary_logloss: 0.412246	valid_1's binary_logloss: 0.414457
[800]	valid_0's binary_logloss: 0.410771	valid_1's binary_logloss: 0.414263
[1000]	valid_0's binary_logloss: 0.409375	valid_1's binary_logloss: 0.414097
[1200]	valid_0's binary_logloss: 0.408044	valid_1's binary_logloss: 0.413993
[1400]	valid_0's binary_logloss: 0.406768	valid_1's binary_logloss: 0.413942
[1600]	valid_0's binary_logloss: 0.405533	valid_1's binary_logloss: 0.413916
Early stopping, best iteration is:
[1518]	valid_0's binary_logloss: 0.406031	valid_1's binary_logloss: 0.413911
[0.41391132240832157]
cv: 0.41391132240832157
Training until validation scores don't improve for 100 rounds.
[200]	valid_0's binary_logloss: 0.415903	valid_1's binary_logloss: 0.417085
[400]	valid_0's binary_loglo

[0.4138514986861429, 0.4150293287481048, 0.41502096663759397, 0.4147727589831188]
cv: 0.4146686382637401
Training until validation scores don't improve for 100 rounds.
[200]	valid_0's binary_logloss: 0.416008	valid_1's binary_logloss: 0.416479
[400]	valid_0's binary_logloss: 0.413759	valid_1's binary_logloss: 0.415374
[600]	valid_0's binary_logloss: 0.412093	valid_1's binary_logloss: 0.415002
[800]	valid_0's binary_logloss: 0.410603	valid_1's binary_logloss: 0.414837
[1000]	valid_0's binary_logloss: 0.409206	valid_1's binary_logloss: 0.414704
[1200]	valid_0's binary_logloss: 0.407865	valid_1's binary_logloss: 0.414604
[1400]	valid_0's binary_logloss: 0.406566	valid_1's binary_logloss: 0.414562
[1600]	valid_0's binary_logloss: 0.405276	valid_1's binary_logloss: 0.414522
Early stopping, best iteration is:
[1509]	valid_0's binary_logloss: 0.405856	valid_1's binary_logloss: 0.41452
[0.4138514986861429, 0.4150293287481048, 0.41502096663759397, 0.4147727589831188, 0.41451976759681625]
cv: 0.

In [37]:
first=2500
split_index = df_fea.sort_values('split',ascending=False)['split'].index.values[:first]
gain_index = df_fea.sort_values('gain',ascending=False)['gain'].index.values[:first]
retain_index = list(set(split_index).intersection(set(gain_index)))
print(len(retain_index))
verify_drop = verify_on_sample(train_csr,train_y,lgb_model,retain_index,n_fold = 5,seed=19,cv_list=[0.41467523398442835])

2387
Training until validation scores don't improve for 100 rounds.
[200]	valid_0's binary_logloss: 0.4162	valid_1's binary_logloss: 0.415942
[400]	valid_0's binary_logloss: 0.413925	valid_1's binary_logloss: 0.414764
[600]	valid_0's binary_logloss: 0.412247	valid_1's binary_logloss: 0.414325
[800]	valid_0's binary_logloss: 0.410777	valid_1's binary_logloss: 0.414124
[1000]	valid_0's binary_logloss: 0.409362	valid_1's binary_logloss: 0.413955
[1200]	valid_0's binary_logloss: 0.408001	valid_1's binary_logloss: 0.413864
[1400]	valid_0's binary_logloss: 0.406724	valid_1's binary_logloss: 0.413796
[1600]	valid_0's binary_logloss: 0.405456	valid_1's binary_logloss: 0.413737
[1800]	valid_0's binary_logloss: 0.40423	valid_1's binary_logloss: 0.413708
[2000]	valid_0's binary_logloss: 0.403014	valid_1's binary_logloss: 0.413695
Early stopping, best iteration is:
[1935]	valid_0's binary_logloss: 0.403403	valid_1's binary_logloss: 0.413686
[0.41368577597892464]
cv: 0.41368577597892464
Training un

In [42]:
first=3500
split_index = df_fea.sort_values('split',ascending=False)['split'].index.values[:first]
gain_index = df_fea.sort_values('gain',ascending=False)['gain'].index.values[:first]
retain_index = list(set(split_index).intersection(set(gain_index)))
print(len(retain_index))
verify_drop = verify_on_sample(train_csr,train_y,lgb_model,retain_index,n_fold = 5,seed=19,cv_list=[0.41467523398442835])

3370
Training until validation scores don't improve for 100 rounds.
[200]	valid_0's binary_logloss: 0.416198	valid_1's binary_logloss: 0.415972
[400]	valid_0's binary_logloss: 0.413916	valid_1's binary_logloss: 0.414822


KeyboardInterrupt: 

In [ ]:
first=4500
split_index = df_fea.sort_values('split',ascending=False)['split'].index.values[:first]
gain_index = df_fea.sort_values('gain',ascending=False)['gain'].index.values[:first]
retain_index = list(set(split_index).intersection(set(gain_index)))
print(len(retain_index))
verify_drop = verify_on_sample(train_csr,train_y,lgb_model,retain_index,n_fold = 5,seed=19,cv_list=[0.41467523398442835])

In [ ]:
first=1000
split_index = df_fea.sort_values('split',ascending=False)['split'].index.values[:first]
gain_index = df_fea.sort_values('gain',ascending=False)['gain'].index.values[:first]
retain_index = list(set(split_index).intersection(set(gain_index)))
print(len(retain_index))
verify_drop = verify_on_sample(train_csr,train_y,lgb_model,retain_index,n_fold = 5,seed=19,cv_list=[0.41467523398442835])

In [ ]:
first=5000
split_index = df_fea.sort_values('split',ascending=False)['split'].index.values[:first]
gain_index = df_fea.sort_values('gain',ascending=False)['gain'].index.values[:first]
retain_index = list(set(split_index).intersection(set(gain_index)))
print(len(retain_index))
verify_drop = verify_on_sample(train_csr,train_y,lgb_model,retain_index,n_fold = 5,seed=19,cv_list=[0.41467523398442835])

In [44]:
first=2000
split_index = df_fea.sort_values('split',ascending=False)['split'].index.values[:first]
gain_index = df_fea.sort_values('gain',ascending=False)['gain'].index.values[:first]
retain_index = list(set(split_index).intersection(set(gain_index)))

In [45]:
train_csr_save=train_csr[:,retain_index]
predict_csr_save=predict_csr[:,retain_index]

In [46]:
save_csr(train_csr_save,'../../data/features/xgb/train_csr_2000')
save_csr(predict_csr_save,'../../data/features/xgb/predict_csr_2000')

data save
indices save
indptr save
save done!
data save
indices save
indptr save
save done!
